# CNN (text + images)

In [2]:
# Import libraries
import numpy as np
import time
import os
import matplotlib
import matplotlib.image as mpimg
import pandas as pd
from torch import optim
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer
import re
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
import spacy
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import cv2 
import multiprocessing as mp
import FunctionsTFM 
import imp
import threading


%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\santi_s09ykpb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\santi_s09ykpb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


First we get the data.

In [3]:
# Train data 
traindata_all = pd.read_csv('C:\\TFM\\Data\\multimodal_train.tsv', sep='\t')
# Validation data 
validata_all = pd.read_csv('C:\\TFM\\Data\\multimodal_validate.tsv', sep='\t')
# Test data 
testdata_all = pd.read_csv('C:\\TFM\\Data\\multimodal_test_public.tsv', sep='\t')

Select a subset of the dataframe with no missing values in the 'title' column.

In [4]:
# Train data with no missing values
train_data = traindata_all[traindata_all['title'].notnull().to_numpy()]
# Validation data with no missing values
valid_data = validata_all[validata_all['title'].notnull().to_numpy()]
# Test data with no missing values
test_data = testdata_all[testdata_all['title'].notnull().to_numpy()]

Separate the dataset into the texts, labels and image names

In [5]:
## Train data
train_news = list(train_data['title'])
train_labels = list(train_data['6_way_label'])
train_images = list(train_data['id'])
## Valid data
valid_news = list(valid_data['title'])
valid_labels = list(valid_data['6_way_label'])
valid_images = list(valid_data['id'])
## Test data
test_news = list(test_data['title'])
test_labels = list(test_data['6_way_label'])
test_images = list(test_data['id'])

Now we add the **'.jpg'** termination to all the image names in the previous lists.

In [6]:
# Train
train_images_final = [image + '.jpg' for image in train_images]
# Validation
valid_images_final = [image + '.jpg' for image in valid_images]
# Test
test_images_final = [image + '.jpg' for image in test_images]


## Preprocessing

First we neet to preprocess the text that we will feed to the neural network. We define a function to preprocess the data. We remove punctuations, numbers and also multiple spaces.

In [9]:
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [10]:
# Remove puntuations and numbers and multiple spaces

train_news_clean_1 = []
valid_news_clean_1 = []
test_news_clean_1 = []
# Train
for new in train_news:
    train_news_clean_1.append(preprocess_text(new))
# Validation
for new in valid_news:
    valid_news_clean_1.append(preprocess_text(new))
# Test
for new in test_news:
    test_news_clean_1.append(preprocess_text(new))

In [11]:
# Initialize  lemmatizer and  stop_words

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english')) 

# Function to remove stopwords
def remove_stopwords_lem(text):
    text = word_tokenize(text)
    # Remove stopwords
    text = [word for word in text if word not in stop_words]
    # Lematization
    lemmatized_text = []
    for word in text:
        word1 = lemmatizer.lemmatize(word, pos = "n")
        word2 = lemmatizer.lemmatize(word1, pos = "v")
        word3 = lemmatizer.lemmatize(word2, pos = ("a"))
        lemmatized_text.append(word3)
        
    text_done = ' '.join(lemmatized_text)
    return text_done

And now we perform stop-words removal and lemmatization

In [12]:
# Stop-words removal and lemmatization
train_stwrd_lem = []
valid_stwrd_lem = []
test_stwrd_lem = []

# Train
for new in train_news_clean_1:
    train_stwrd_lem.append(remove_stopwords_lem(new))
# Validation
for new in valid_news_clean_1:
    valid_stwrd_lem.append(remove_stopwords_lem(new))
# Test
for new in test_news_clean_1:
    test_stwrd_lem.append(remove_stopwords_lem(new))

And now we tokenize the news. For that we train a tokenizer using all the vocabulary.

In [13]:
news_all = train_stwrd_lem + valid_stwrd_lem + test_stwrd_lem

tokenizer = Tokenizer(num_words = 128022)
tokenizer.fit_on_texts(news_all)

# Tokenize news

# Train
train_tokenized = tokenizer.texts_to_sequences(train_stwrd_lem)
# Validation
valid_tokenized = tokenizer.texts_to_sequences(valid_stwrd_lem)
# Test
test_tokenized = tokenizer.texts_to_sequences(test_stwrd_lem)

Obtain the vocabulary length

In [14]:
print("Vocabulary length: ", len(tokenizer.word_index))

Vocabulary length:  109845


Now we pad the sequences of numbers generated by the tokenizer. We select **15** as the lenght of the padded/truncated sequences since, as can be seen in the 'CNN.ipynb' and in the  'BiLSTM.ipynb' codes, almos all news are shorter than this, so very little information will be lost.

In [16]:
# Pad/truncate the tokenized news

# Train
train_tokenized_pad = pad_sequences(train_tokenized, maxlen = 15, truncating = 'post', padding = 'post')
# Validation
valid_tokenized_pad = pad_sequences(valid_tokenized, maxlen = 15, truncating = 'post', padding = 'post')
# Test
test_tokenized_pad = pad_sequences(test_tokenized, maxlen = 15, truncating = 'post', padding = 'post')

## Training the model

We create a function to load the word embeddings and another one to create the embedding matrix.

In [17]:
# Function to load the word embeddings

def load_embedd(filename):
    words = []
    vectors = []
    file = open(filename,'r', encoding="utf8")
    for line in file.readlines():
       row = line.split(' ')
       vocab = row[0]
       embd = row[1:len(row)]
       embd[-1] = embd[-1].rstrip()
       embd = list(map(float,embd)) # convert string to float
       words.append(vocab)
       vectors.append(embd)
    file.close()
    return words,vectors

In [18]:
# Function embed matrix

def embed_matx(word_index, vocab, embeddings, length_vocab, length_embedding):
    embedding_matrix = np.zeros((length_vocab +1, length_embedding))
    for word, i in word_index.items():
        if word in vocab:
            idx = vocab.index(word)
            vector =  embeddings[idx]
            embedding_matrix[i] = vector
        if i == length_vocab:
            break
    return embedding_matrix

#### GloVe (300 d)

Obtain the vocabulary and the vectors of the embedding matrix. We use GloVe embeddings of dimension 300.

In [19]:
vocab_gv_300, vectors_gv_300 = load_embedd(filename = "H:\\TFM\\Glove_embeddings\\glove.6B.300d.txt")

Create the embedding matrix.

In [20]:
word_index = tokenizer.word_index
# Embedding matrix
embedding_matrix_gv_300 = embed_matx(word_index = word_index, vocab = vocab_gv_300, embeddings = vectors_gv_300, 
                             length_vocab = 109845, length_embedding = 300)

In [21]:
# Get names of available images
all_images = os.listdir("C:\\TFM\\Data\\public_image_set")

# Path to the images folder

PATH = "C:\\TFM\\Data\\public_image_set\\"

In order to feed the images to the CNN these need to be of the same size therefore we will have to pad the images with zeros or truncate them so that all are of the size **560 x 560**. In order to achieve that we define the following function.

In [22]:
# Funtion to pad/truncate images to shape 560 x 560

def pad_images(image):
     img_reshaped = np.zeros((3, 560, 560))
     if image.shape[0] > 560 and image.shape[1] > 560:
       img_reshaped[0, :, :] = image[0:560,0:560,0]
       img_reshaped[1, :, :] = image[0:560,0:560,1]
       img_reshaped[2, :, :] = image[0:560,0:560,2]
     if image.shape[0] > 560 and image.shape[1] <= 560:
       img_reshaped[0, :, 0:image.shape[1]] = image[0:560,:,0]
       img_reshaped[1, :, 0:image.shape[1]] = image[0:560,:,1]
       img_reshaped[2, :, 0:image.shape[1]] = image[0:560,:,2]
     if image.shape[0] <= 560 and image.shape[1] > 560:
       img_reshaped[0, 0:image.shape[0], :] = image[:,0:560,0]
       img_reshaped[1, 0:image.shape[0], :] = image[:,0:560,1]
       img_reshaped[2, 0:image.shape[0], :] = image[:,0:560,2]
     if image.shape[0] < 560 and image.shape[1] < 560:
       img_reshaped[0, 0:image.shape[0], 0:image.shape[1]] = image[:,:,0]
       img_reshaped[1, 0:image.shape[0], 0:image.shape[1]] = image[:,:,1]
       img_reshaped[2, 0:image.shape[0], 0:image.shape[1]] = image[:,:,2]
     return img_reshaped

## Model

We also define the different layers that we will be using for  the model.

In [25]:
# Create model

class CNN_title_images(nn.Module):
    
  def __init__(self,image_dimx,nlabels):
    
     super().__init__()
        
     # Embedding layer 
     self.embedding = nn.Embedding(num_embeddings =109846, embedding_dim = 300)
     self.embedding.weight = nn.Parameter(torch.from_numpy(embedding_matrix_gv_300), requires_grad = False)

     # Convolutional layers (for the images)
     self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0)
     self.conv2 = nn.Conv2d(in_channels=6, out_channels=3, kernel_size=5, stride=1, padding=0)

     # Convolutional layer (for the titles)
     self.filter_sizes = [2,3,4,5]
     self.num_filters = 50
     self.convs_concat = nn.ModuleList([nn.Conv2d(1, self.num_filters, (K, 300)) for K in self.filter_sizes])

     # Pooling layer
     self.maxlayer1 = nn.MaxPool2d(2, 2)

     # Linear layers
     self.linear1 = nn.Linear(200 + 3*137*137,256)
     self.linear2 = nn.Linear(256,nlabels)
     self.linear3 = nn.Linear(128,nlabels)

     # Relu activation
     self.relu = nn.ReLU()

     # Logsoftmax function
     self.logsoftmax = nn.LogSoftmax(dim=1) 

     # Dimension of the image at the output of the second convolutional layer
     self.final_dim = int(((image_dimx-4)/2-4)/2)
        
  def forward(self, x_image, x_title):
        # Pass the image tensor through the CNN operations
        x_image = self.conv1(x_image) 
        x_image = self.relu(x_image) 
        x_image = self.maxlayer1(x_image)
        x_image = self.conv2(x_image)
        x_image = self.relu(x_image)
        x_image = self.maxlayer1(x_image)
        x_image = x_image.view(x_image.shape[0],self.final_dim*self.final_dim*3)
        # Pass the text tensor through the different operations
        x_title = self.embedding(x_title)
        x_title = x_title.unsqueeze(1)
        x_title = [F.relu(conv(x_title.float())).squeeze(3) for conv in self.convs_concat]
        x_title = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x_title] 
        x_title = torch.cat(x_title, 1)
        # Join text and image output
        x = torch.cat([x_image,x_title ],1)
        # Pass result through linear layers
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.logsoftmax(x) 
        return x



### Optimal number of epochs

We instantiate the class

In [26]:
# Instantiate the model
model_CNN_image_text = CNN_title_images(image_dimx = 560 ,nlabels = 6 )

### Optimal number of epochs

We train the model for **1**, **2** and **3** epochs and we select the number of epochs that results in the highest validation accuracy.

**NOTE**: run the code in the cell bellow once for 1 epochs, twice for 2 epochs and trice for 3 epochs. In each case, run also the cell that follows the cell below to obtain the corresponding validation accuracy.

In [32]:
# Create counter to be used in the loop
counter = 0
# List to store the images
list_images = []
# List to store the titles
list_titles = []
# List to store the labels
list_labels = []
# To count the number of batches
repetitions_train = 0
repetitions_valid = 0
# Error function
criterion = nn.NLLLoss()
# Optimizer
optimizer = torch.optim.Adam(model_CNN_image_text.parameters(), lr = 0.001)
# To store train accuracy
accuracy_train = 0
# To store valation accuracy
accuracy_valid = 0
#  Loop to train the model

images_tensor = torch.zeros(60,3,560,560)
titles_tensor = torch.zeros(60,15)
labels_tensor = torch.zeros(60)

for i in range(0, len(train_images_final)):
    
    # First check if image is available
    img = cv2.imread(PATH + train_images_final[i])   
        
    if type(img) is not type(None):
                          
        counter += 1
        
        # Pad it with zeros or truncate it to obtain 560 x 560 shape
        
        img_padded = pad_images(img)
        
        # Store image and corresponding title and label 
        images_tensor[counter-1, :,:,:] = torch.from_numpy(img_padded)
        titles_tensor[counter-1, :] = torch.from_numpy(train_tokenized_pad[i]).int()
        labels_tensor[counter - 1] = train_labels[i]    
        
        # When 'batch_size' obervations are stored create DataLoader objects 
        
        if counter%60 == 0:
            
           repetitions_train += 1
           print("Batch interaction: ", repetitions_train)

           # Reset gradients
           optimizer.zero_grad() 

           # Pass image and text through the different layers
            
           out = model_CNN_image_text.forward(images_tensor, titles_tensor.long())         
            
           # Compute loss
        
           loss = criterion(out,labels_tensor.long())
            
           # Backpropagation
           
           loss.backward()
        
           # Optimize parameters
            
           optimizer.step()
            
           # Obtain number of correct predictions and store accuracy
            
           top_p, top_class = out.topk(1, dim=1) 
           equals = (top_class == labels_tensor.view(images_tensor.shape[0], 1))
           accuracy_train += torch.mean(equals.type(torch.FloatTensor))
        
           images_tensor = torch.zeros(60,3,560,560)
           titles_tensor = torch.zeros(60,15)
           labels_tensor = torch.zeros(60)
            
           counter = 0 
                
           # Every 100 repetitions print accuracy
           if repetitions_train%100 == 0:
                print("Training accuracy:", accuracy_train/repetitions_train)
            

Batch interaction:  1
Batch interaction:  2
Batch interaction:  3
Batch interaction:  4
Batch interaction:  5
Batch interaction:  6
Batch interaction:  7
Batch interaction:  8
Batch interaction:  9
Batch interaction:  10
Batch interaction:  11
Batch interaction:  12
Batch interaction:  13
Batch interaction:  14
Batch interaction:  15
Batch interaction:  16
Batch interaction:  17
Batch interaction:  18
Batch interaction:  19
Batch interaction:  20
Batch interaction:  21
Batch interaction:  22
Batch interaction:  23
Batch interaction:  24
Batch interaction:  25
Batch interaction:  26
Batch interaction:  27
Batch interaction:  28
Batch interaction:  29
Batch interaction:  30
Batch interaction:  31
Batch interaction:  32
Batch interaction:  33
Batch interaction:  34
Batch interaction:  35
Batch interaction:  36
Batch interaction:  37
Batch interaction:  38
Batch interaction:  39
Batch interaction:  40
Batch interaction:  41
Batch interaction:  42
Batch interaction:  43
Batch interaction:  

After each epoch we evaluate the model over the validation partition.

In [33]:
counter = 0

images_tensor = torch.zeros(60,3,560,560)
titles_tensor = torch.zeros(60,15)
labels_tensor = torch.zeros(60)
accuracy_valid = 0

# Validation pass

with torch.no_grad():
    
 for i in range(len(valid_images_final)):
    
    # First check if image is available
    
    img = cv2.imread(PATH + valid_images_final[i])
        
    if type(img) is not type(None):
        
        # Add 1 to counter
        
        counter += 1
               
        # Pad it with zeos to obtain 560 x 560 shape
        
        img_padded = pad_images(img)
        
        # Store image and corresponding title and label 
        images_tensor[counter-1, :,:,:] = torch.from_numpy(img_padded)
        titles_tensor[counter-1, :] = torch.from_numpy(valid_tokenized_pad[i]).int()
        labels_tensor[counter - 1] = valid_labels[i] 
         
        
        if counter%60 == 0:
            
           repetitions_valid += 1

           # Pass image and text through the different layers
            
           out = model_CNN_image_text.forward(images_tensor, titles_tensor.int())         
                 
           # Obtain number of correct predictions and store accuracy
            
           top_p, top_class = out.topk(1, dim=1) 
           equals = (top_class == labels_tensor.view(images_tensor.shape[0], 1))
           accuracy_valid += torch.mean(equals.type(torch.FloatTensor))
        
           images_tensor = torch.zeros(60,3,560,560)
           titles_tensor = torch.zeros(60,15)
           labels_tensor = torch.zeros(60)
            
           counter = 0 
           

print("Validation accuracy:", accuracy_valid/repetitions_valid)

Validation accuracy: tensor(0.8703)


The following table shows the accuracy that we have obtained for each number of epochs.

| Epochs | Accuracy |
|--------|----------|
|   1    |  0.8635  |
|   2    |  0.8707  |
|   3    |  0.8703  |

As we can see the highest accuracy is obtained when we train the model for **2** epochs so this is the number of epochs that we will use.

## Testing performance

### Training with train + validation data

We join the train and validation data and we train the model with it.

In [34]:
# Join train and validation sequences
train_valid_tokenized_pad = np.concatenate((train_tokenized_pad, valid_tokenized_pad), axis = 0)
# Join train and validation labels
train_valid_labels = train_labels + valid_labels
# Join images names
train_valid_images_final = train_images_final + valid_images_final

Train the model.

In [35]:
# Instantiate model
model_CNN_image_text_final = CNN_title_images(image_dimx = 560 ,nlabels = 6 )

The following cell has to be run 2 times, one for each epoch.

In [38]:
# Create counter to be used in the loop
counter = 0
# To count the number of batches
repetitions_train = 0
repetitions_valid = 0
# Error function
criterion = nn.NLLLoss()
# Optimizer
optimizer = torch.optim.Adam(model_CNN_image_text_final.parameters(), lr = 0.001)
# To store train accuracy
accuracy_train = 0
# To store valation accuracy
accuracy_valid = 0


images_tensor = torch.zeros(60,3,560,560)
titles_tensor = torch.zeros(60,15)
labels_tensor = torch.zeros(60)

for i in range(len(train_valid_images_final)):
    
    # First check if image is available
    img = cv2.imread(PATH + train_valid_images_final[i])   
        
    if type(img) is not type(None):
                          
        counter += 1
        
        # Pad it with zeros or truncate it to obtain 560 x 560 shape
        
        img_padded = pad_images(img)
        
        # Store image and corresponding title and label 
        images_tensor[counter-1, :,:,:] = torch.from_numpy(img_padded)
        titles_tensor[counter-1, :] = torch.from_numpy(train_valid_tokenized_pad[i]).int()
        labels_tensor[counter - 1] = train_valid_labels[i]    
        
        
        if counter%60 == 0:
            
           repetitions_train += 1
           print("Batch interaction: ", repetitions_train)

           # Reset gradients
           optimizer.zero_grad() 

           # Pass image and text through the different layers
            
           out = model_CNN_image_text_final.forward(images_tensor, titles_tensor.long())         
            
           # Compute loss
        
           loss = criterion(out,labels_tensor.long())
            
           # Backpropagation
           
           loss.backward()
        
           # Optimize parameters
            
           optimizer.step()
            
           # Obtain number of correct predictions and store accuracy
            
           top_p, top_class = out.topk(1, dim=1) 
           equals = (top_class == labels_tensor.view(images_tensor.shape[0], 1))
           accuracy_train += torch.mean(equals.type(torch.FloatTensor))
        
           images_tensor = torch.zeros(60,3,560,560)
           titles_tensor = torch.zeros(60,15)
           labels_tensor = torch.zeros(60)
            
           counter = 0 
                
           # Every 100 repetitions print accuracy
           if repetitions_train%100 == 0:
                print("Training accuracy:", accuracy_train/repetitions_train)


Batch interaction:  1
Batch interaction:  2
Batch interaction:  3
Batch interaction:  4
Batch interaction:  5
Batch interaction:  6
Batch interaction:  7
Batch interaction:  8
Batch interaction:  9
Batch interaction:  10
Batch interaction:  11
Batch interaction:  12
Batch interaction:  13
Batch interaction:  14
Batch interaction:  15
Batch interaction:  16
Batch interaction:  17
Batch interaction:  18
Batch interaction:  19
Batch interaction:  20
Batch interaction:  21
Batch interaction:  22
Batch interaction:  23
Batch interaction:  24
Batch interaction:  25
Batch interaction:  26
Batch interaction:  27
Batch interaction:  28
Batch interaction:  29
Batch interaction:  30
Batch interaction:  31
Batch interaction:  32
Batch interaction:  33
Batch interaction:  34
Batch interaction:  35
Batch interaction:  36
Batch interaction:  37
Batch interaction:  38
Batch interaction:  39
Batch interaction:  40
Batch interaction:  41
Batch interaction:  42
Batch interaction:  43
Batch interaction:  

In [ ]:
# Save model
torch.save(model_CNN_image_text_final.state_dict(), 'model_with_images_test.pt')

### Model evaluation

In [39]:
counter = 0

images_tensor = torch.zeros(60,3,560,560)
titles_tensor = torch.zeros(60,15)
labels_tensor = torch.zeros(60)

predictions = np.array([])
labels_test = np.array([])

# Validation pass

with torch.no_grad():
    
 for i in range(len(test_images_final)):
    
    # First check if image is available
    
    img = cv2.imread(PATH + test_images_final[i])
        
    if type(img) is not type(None):
        
        # Add 1 to counter
        
        counter += 1
               
        # Pad it with zeos to obtain 560 x 560 shape
        
        img_padded = pad_images(img)
        
        # Store image and corresponding title and label 
        images_tensor[counter-1, :,:,:] = torch.from_numpy(img_padded)
        titles_tensor[counter-1, :] = torch.from_numpy(test_tokenized_pad[i]).int()
        labels_tensor[counter - 1] = test_labels[i] 
        
        if counter%60 == 0:

           # Pass image and text through the different layers
            
           out = model_CNN_image_text_final.forward(images_tensor, titles_tensor.int())         
                 
           # Obtain predictions
            
           top_p, top_class = out.topk(1, dim=1) 
           predictions = np.concatenate((predictions, top_class.numpy().reshape(-1)))
           labels_test =  np.concatenate((labels_test, labels_tensor.numpy()))
        
           images_tensor = torch.zeros(60,3,560,560)
           titles_tensor = torch.zeros(60,15)
           labels_tensor = torch.zeros(60)
            
           counter = 0 
           

Finally we can obtain the results of the model evaluation over the test data.

In [40]:
print(classification_report(labels_test, predictions))

              precision    recall  f1-score   support

         0.0       0.85      0.88      0.86     23489
         1.0       0.82      0.72      0.77      3513
         2.0       0.77      0.76      0.76     11290
         3.0       0.46      0.25      0.32      1223
         4.0       1.00      1.00      1.00     17400
         5.0       0.75      0.79      0.77      2305

    accuracy                           0.87     59220
   macro avg       0.77      0.73      0.75     59220
weighted avg       0.86      0.87      0.86     59220



In [41]:
# Classification report without 0 label
print(classification_report(np.array(labels_test).reshape(len(labels_test),1),predictions, labels = [1,2,3,4,5]))

              precision    recall  f1-score   support

           1       0.82      0.72      0.77      3513
           2       0.77      0.76      0.76     11290
           3       0.46      0.25      0.32      1223
           4       1.00      1.00      1.00     17400
           5       0.75      0.79      0.77      2305

   micro avg       0.88      0.86      0.87     35731
   macro avg       0.76      0.70      0.72     35731
weighted avg       0.87      0.86      0.86     35731



In [42]:
# Confusion matrix
print(confusion_matrix(np.array(labels_test).reshape(len(labels_test),1),predictions))

[[20703   283  2048   192     4   259]
 [  604  2540   262    43     9    55]
 [ 2229   175  8564    76     7   239]
 [  618    46   190   306     4    59]
 [    4    10    17     0 17367     2]
 [  275    53   110    46     0  1821]]
